In [2]:
# Motion detection
import imutils, time, cv2

# get the frame from the camera
video = cv2.VideoCapture(0)

# set first frame as None initially
first_frame = None

# read frames continuously
while True:
    # read the frame
    check, frame = video.read()
    
    # convert to gray scale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # add some noise using Gaussian blur 
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)
    
    # set the firstly captured frame
    if first_frame is None:
        first_frame = gray_frame
        continue
    
    # find the difference between the first frame and subsequent frame
    frame_delta = cv2.absdiff(first_frame, gray_frame)
    
    # set a threshold as 60. Only if frame threshold is above 60, detection is possible
    threshold = cv2.threshold(frame_delta, 60, 255, cv2.THRESH_BINARY)[1]
    
    # dilate to increase object area, removes noise and joins broken parts of object
    # iterations - determine how much you want to erode/dilate a given image
    threshold = cv2.dilate(threshold, None, iterations=0)

    # removes insignificant objects like shadows, small objects etc and get the contours
    contours = cv2.findContours(threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    
    # loop over the contours
    for c in contours:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 2000:
            continue
        # get the bounding rectangle in the contour
        (x, y, w, h) = cv2.boundingRect(c)
        # draw rectangle in the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
         
    # shows the image
    cv2.imshow("video", frame)
    cv2.imshow("gray", gray_frame)
    
    # waits till z is pressed
    key = cv2.waitKey(1) & 0xff
    if key == ord('z'):
        break
        
# close the video
video.release()
cv2.destroyAllWindows()

In [4]:
# Monitor motion detection time
import pandas, cv2, time, imutils
from datetime import datetime

# initialize status list and times to capture motion time
first_frame = None
status_list = [None, None]
times = []
df = pandas.DataFrame(columns=["Start", "End"])
video = cv2.VideoCapture(0)

while True:
    check, frame = video.read()
    
    # the status will be 0 during capture
    status = 0
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)
    if first_frame is None:
        first_frame = gray_frame
        continue
        
    frame_delta = cv2.absdiff(first_frame, gray_frame)
    threshold = cv2.threshold(frame_delta, 60, 255, cv2.THRESH_BINARY)[1]
    threshold = cv2.dilate(threshold, None, iterations=0)
    contours = cv2.findContours(threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    
    # loop over the contours
    for c in contours:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 2000:
            continue
        
        # set the status to 1 when motion is found
        status = 1
        
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
    
    # add the status in the status_list
    status_list.append(status)
    status_list = status_list[-2:]
    
    # if motion found 0->1 or 1->0 in the last two frames, capture the time and record it
    if (status_list[-1] == 0 and status_list[-2] == 1) or (status_list[-1] == 1 and status_list[-2] == 0):
        times.append(datetime.now())
        
    # show the motion detection
    cv2.imshow("video", frame)
    cv2.imshow("gray", gray_frame)
    
    # show video till z is pressed
    key = cv2.waitKey(1) & 0xff
    if key == ord('z'):
        break

# print the status list and times 
print(status_list)
print(times)

# store in a data frame the times of motion detected
for ind in range(0, len(times), 2):
    df = df.append({"Start": times[ind], "End": times[ind + 1]}, ignore_index=True)

# write into a csv file
df.to_csv("Motion_detection_times.csv")

# close the video
video.release()
cv2.destroyAllWindows() 

[0, 0]
[datetime.datetime(2021, 6, 5, 13, 16, 1, 155751), datetime.datetime(2021, 6, 5, 13, 16, 5, 28658), datetime.datetime(2021, 6, 5, 13, 16, 9, 315769), datetime.datetime(2021, 6, 5, 13, 16, 9, 379690), datetime.datetime(2021, 6, 5, 13, 16, 9, 507634), datetime.datetime(2021, 6, 5, 13, 16, 10, 115762), datetime.datetime(2021, 6, 5, 13, 16, 11, 123690), datetime.datetime(2021, 6, 5, 13, 16, 12, 115638), datetime.datetime(2021, 6, 5, 13, 16, 12, 260816), datetime.datetime(2021, 6, 5, 13, 16, 12, 323868), datetime.datetime(2021, 6, 5, 13, 16, 14, 595515), datetime.datetime(2021, 6, 5, 13, 16, 14, 931540), datetime.datetime(2021, 6, 5, 13, 16, 15, 61638), datetime.datetime(2021, 6, 5, 13, 16, 15, 859891)]
